# Convolutional Neural Network using MNIST

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="6"


## Loading of the data and placeholder definitions

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

n_classes = 10
batch_size = 128

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')
keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Functions for Convolution and Maxpooling

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x,W, strides = [1,1,1,1], padding = 'SAME') # Takes 1X1 pixels so that no pixels are skipped
                                                                        # Padding is done to cover the edges
    
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding  = 'SAME')# Movement is 2X2 at a time
                                                                                            # Padding is done to cover the edges
                                                                                            # ksize is size of window and strides are for movement


## Network Definition and construction

In [4]:
def cnn_model(x):
    Weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])), # 5X5 Convolution taking 1 input unit and producing 32 features
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
               'out':tf.Variable(tf.random_normal([1024,n_classes])),
              }
                      
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])), # 5X5 Convolution taking 1 input unit and producing 32 features
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes])),
             }
    
    x = tf.reshape(x, shape = [-1,28,28,1])
    
    conv1 = tf.nn.relu(conv2d(x, Weights['W_conv1'])+biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, Weights['W_conv2'])+biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, Weights['W_fc'])+biases['b_fc'])
    
    fc = tf.nn.dropout(fc, keep_rate)
    
    output = tf.matmul(fc, Weights['out']) + biases['out']

    return output

## Training of the Network

In [5]:
def train_neural_network(x):
    prediction = cnn_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    gpu_options = tf.GPUOptions(allow_growth=True)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)

Epoch 0 completed out of 10 loss: 2929267.198425293
Epoch 1 completed out of 10 loss: 590434.5893859863
Epoch 2 completed out of 10 loss: 319759.05463409424
Epoch 3 completed out of 10 loss: 208352.64449310303
Epoch 4 completed out of 10 loss: 143349.7575750351
Epoch 5 completed out of 10 loss: 105877.07577514648
Epoch 6 completed out of 10 loss: 83919.23741149902
Epoch 7 completed out of 10 loss: 61186.3710179925
Epoch 8 completed out of 10 loss: 49046.925503730774
Epoch 9 completed out of 10 loss: 44563.42469406128
Accuracy: 0.9655
